# Imports

## Import des libraries

In [3]:
import pandas as pd
import numpy as np 


# Import des données

In [4]:
df = pd.read_csv("data/df_enc.csv")

In [5]:
df

,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head_Child,...,job_type_Dont Know/Refuse to answer,job_type_Farming and Fishing,job_type_Formally employed Government,job_type_Formally employed Private,job_type_Government Dependent,job_type_Informally employed,job_type_No Income,job_type_Other Income,job_type_Remittance Dependent,job_type_Self employed
0,1,2018,uniqueid_1,1,1,1,3,24,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,2018,uniqueid_2,0,1,0,5,70,1,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1,2018,uniqueid_3,1,2,1,5,26,2,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1,2018,uniqueid_4,0,1,1,5,34,1,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,2018,uniqueid_5,0,2,0,8,26,2,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23519,4,2018,uniqueid_2113,0,1,1,4,48,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
23520,4,2018,uniqueid_2114,0,1,1,2,27,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
23521,4,2018,uniqueid_2115,0,1,1,5,27,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
23522,4,2018,uniqueid_2116,0,2,1,7,30,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# Baseline model ( Dummy Classifier )

I need to have a benchmark for my future iteration, so I create a simple model to begin with

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

In [9]:
X = df.drop(["uniqueid", "bank_account"], axis = 1)
y = df["bank_account"]

In [12]:
# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=2)

In [13]:
dummy_clf = DummyClassifier(strategy="most_frequent")

# fit the modem
dummy_clf.fit(X_train, y_train)

# calculate the score
dummy_clf.score(X_test, y_test)

0.8634173986965146

# Cross Validation

I will use cross validate method for avoid overfitting and having more reliable results

In [16]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

In [17]:
# Instanciate model
dummy_clf = DummyClassifier(strategy="most_frequent")

# 5-Fold Cross validate model
cv_results = cross_validate(dummy_clf, X, y, cv=5)

# obtain the mean of scores
cv_results['test_score'].mean()

0.8592076203488833